In [1]:
import h5py
import pandas as pd 
import numpy as np 
import torch 
import os 
from torch.utils.data import DataLoader,Dataset


In [2]:
microarray_datasets = ['BRCA_RNASeq2GeneNorm', 
                      'BRCA_mRNAArray', 
                      'COAD_RNASeq2GeneNorm',
                      'COAD_mRNAArray',
                      'KIRC_RNASeq2GeneNorm',
                      'KIRC_mRNAArray',
                      'KIRP_RNASeq2GeneNorm',
                      'KIRP_mRNAArray',
                      'LGG_RNASeq2GeneNorm',
                      'LGG_mRNAArray',
                      'LUAD_RNASeq2GeneNorm',
                      'LUAD_mRNAArray',
                      'READ_RNASeq2GeneNorm',
                      'READ_mRNAArray',
                      'UCEC_RNASeq2GeneNorm',
                      'UCEC_mRNAArray']

csv_files = ["~/Downloads/TCGA_mRNA_RNA/BRCA/BRCA_RNASeq2GeneNorm.csv",
"~/Downloads/TCGA_mRNA_RNA/BRCA/BRCA_mRNAArray.csv",
"~/Downloads/TCGA_mRNA_RNA/COAD/COAD_RNASeq2GeneNorm.csv",
"~/Downloads/TCGA_mRNA_RNA/COAD/COAD_mRNAArray.csv",
"~/Downloads/TCGA_mRNA_RNA/KIRC/KIRC_RNASeq2GeneNorm.csv",
"~/Downloads/TCGA_mRNA_RNA/KIRC/KIRC_mRNAArray.csv",
"~/Downloads/TCGA_mRNA_RNA/KIRP/KIRP_RNASeq2GeneNorm.csv",
"~/Downloads/TCGA_mRNA_RNA/KIRP/KIRP_mRNAArray.csv",
"~/Downloads/TCGA_mRNA_RNA/LGG/LGG_RNASeq2GeneNorm.csv",
"~/Downloads/TCGA_mRNA_RNA/LGG/LGG_mRNAArray.csv",
"~/Downloads/TCGA_mRNA_RNA/LUAD/LUAD_RNASeq2GeneNorm.csv",
"~/Downloads/TCGA_mRNA_RNA/LUAD/LUAD_mRNAArray.csv",
"~/Downloads/TCGA_mRNA_RNA/READ/READ_RNASeq2GeneNorm.csv",
"~/Downloads/TCGA_mRNA_RNA/READ/READ_mRNAArray.csv",
"~/Downloads/TCGA_mRNA_RNA/UCEC/UCEC_RNASeq2GeneNorm.csv",
"~/Downloads/TCGA_mRNA_RNA/UCEC/UCEC_mRNAArray.csv"]

In [3]:
microarray_data = [pd.read_csv(csv_files[i], index_col=0).fillna(0) for i in range(len(csv_files))]

In [4]:
## Training data
microarray_rna = h5py.File('microarray_rna.h5', mode='r') 
u_cancers = list(microarray_rna['microarray_rna/train'])

X = np.vstack(list([microarray_rna['microarray_rna/train/'+c] for c in u_cancers]))
cancer = np.array([c for c in u_cancers for i in range(microarray_rna['microarray_rna/train/'+c].shape[0])])

# Labels
categories = pd.factorize(cancer)
y = categories[0]
y

array([ 0,  0,  0, ..., 15, 15, 15])

In [51]:
u_cancers = list(microarray_rna['microarray_rna/test'])

X_test = np.vstack(list([microarray_rna['microarray_rna/test/'+c] for c in u_cancers]))
cancer_test = np.array([c for c in u_cancers for i in range(microarray_rna['microarray_rna/test/'+c].shape[0])])
categories_test = pd.factorize(cancer_test)
y_test = categories_test[0]

In [7]:
#import stuff 
import yaml
import os
import sys
import shutil
import numpy as np
import torch
import h5py

from itertools import cycle

from torch.backends import cudnn
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable, grad

from src.data import LoadDataset
from src.ufdn import LoadModel

#We'll need to modify this for 3+ domains
from src.util import vae_loss, calc_gradient_penalty, interpolate_vae_3d

from tensorboardX import SummaryWriter 

/Users/kompa/anaconda3/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [8]:
#set this to the path of this yaml file 
#get it from my github branch 
config_path = 'config/microarray_rna.yaml'

In [9]:
conf = yaml.load(open(config_path,'r'))
exp_name = conf['exp_setting']['exp_name']
#img_size is only used in conv nets
#originally it was 64
img_size = conf['exp_setting']['img_size']
#20,501 img_depth
img_depth = conf['exp_setting']['img_depth']
domains = conf['exp_setting']['domains']
number_of_domains = len(domains)


data_root = conf['exp_setting']['data_root']
batch_size = conf['trainer']['batch_size']


enc_dim = conf['model']['vae']['encoder'][-1][1] #latent space dimension #100
code_dim = conf['model']['vae']['code_dim'] #number of domains #currently 3 
vae_learning_rate = conf['model']['vae']['lr'] #learning rate #10e-4
vae_betas = tuple(conf['model']['vae']['betas']) #used for adam optimizer


/Users/kompa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [10]:
#load the model in a blank form 
vae = LoadModel('vae',conf['model']['vae'],img_size,img_depth)

In [85]:
#load in the trained params
#put statedict.pt in the same directory as this ipynb 
vae.load_state_dict(torch.load('/Users/kompa/Downloads/40000.vae', map_location='cpu'))

In [86]:
vae.eval()

UFDN(
  (enc_0): Sequential(
    (0): Linear(in_features=16146, out_features=500, bias=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (enc_mu): Sequential(
    (0): Linear(in_features=500, out_features=100, bias=True)
  )
  (enc_logvar): Sequential(
    (0): Linear(in_features=500, out_features=100, bias=True)
  )
  (dec_0): Sequential(
    (0): Linear(in_features=116, out_features=500, bias=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (dec_1): Sequential(
    (0): Linear(in_features=500, out_features=16146, bias=True)
  )
)

In [38]:
def translate_to_domain(x, vae, domain): 
    '''Takes input x, encodes it via vae and 
       decodes it to the new domain (numbered by cancer_vae)
    '''
    # Encode to latent space
    enc = vae(Variable(torch.FloatTensor(x).unsqueeze(0)),return_enc=True).cpu().data.numpy().ravel()
    # Domain vectors (one for each domain)
    alphas = np.identity(16)
    # The domain we are translating to
    alpha = Variable(torch.FloatTensor(alphas[domain,:] ).unsqueeze(0).expand((1, 16)))
    
    # Decode to new domain 
    dec = vae.decode(Variable(torch.FloatTensor(enc).unsqueeze(0)), alpha).data.numpy().ravel() 
    
    return(dec)

def partial_translate_to_domain(x, vae, original_domain, domain, fraction): 
    '''Takes input x, encodes it via vae and 
       decodes it to the new domain (numbered by cancer_vae)
    '''
    # Encode to latent space
    enc = vae(Variable(torch.FloatTensor(x).unsqueeze(0)),return_enc=True).cpu().data.numpy().ravel()
    # Domain vectors (one for each domain)
    alphas = np.identity(16)
    
    alphas[domain,original_domain] = 1.0 - fraction
    
    alphas[domain, domain] = float(fraction)
    
    alpha = Variable(torch.FloatTensor(alphas[domain,:] ).unsqueeze(0).expand((1, 16)))
    
    # Decode to new domain 
    dec = vae.decode(Variable(torch.FloatTensor(enc).unsqueeze(0)), alpha).data.numpy().ravel() 
    
    return(dec)
    
def translate_dataset(X, vae, domain):
    return(np.vstack(list([translate_to_domain(x, vae, domain) for x in X])))

def partial_translate_dataset(X, vae, original_domain, domain, fraction):
    return(np.vstack(list([partial_translate_to_domain(x, vae, original_domain, domain, fraction) for x in X])))

def encode_sample(x, vae):
    return(vae(Variable(torch.FloatTensor(x).unsqueeze(0)),return_enc=True).cpu().data.numpy().ravel())

def encode_dataset(X, vae):
    return(np.vstack(list([encode_sample(x, vae) for x in X])))

In [81]:
np.linalg.norm(translate_to_domain(X[805], vae, 1)-X[1266])/np.linalg.norm(X[1266])

1220.6242301437592

In [76]:
train_encoded = encode_dataset(X, vae)

In [77]:
test_encoded = encode_dataset(X_test, vae)

In [78]:
# save these encodings 
np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/train_X_encoded_385.csv', train_encoded, delimiter=',')
np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/test_X_encoded_385.csv', test_encoded, delimiter=',')

In [30]:
# bind corresponding samples 
train_samples = np.concatenate(list([np.loadtxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/train_'+c+'.txt', dtype='str') for c in u_cancers]), axis=0)
test_samples = np.concatenate(list([np.loadtxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/test_'+c+'.txt', dtype='str') for c in u_cancers]), axis=0)

In [35]:
# save these sapmle lists 
np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/train_X_samples.csv', train_samples, delimiter=',', fmt='%s')
np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/test_X_samples.csv', test_samples, delimiter=',', fmt='%s')

In [54]:
for idx, cancer in enumerate(u_cancers): 
    print('Interpolating... '+cancer)
    domain = (-1)**(idx%2!=0)+idx
    
    translated_training_data = translate_dataset(X[np.where(y==idx)], vae, domain)
    translated_test_data = translate_dataset(X_test[np.where(y_test==idx)], vae, domain)
    
    np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/train_X_'+cancer+'_interpolated.csv', translated_training_data, delimiter=',')
    np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/test_X_'+cancer+'_interpolated.csv', translated_test_data, delimiter=',')

Interpolating... BRCA_RNASeq2GeneNorm
Interpolating... BRCA_mRNAArray
Interpolating... COAD_RNASeq2GeneNorm
Interpolating... COAD_mRNAArray
Interpolating... KIRC_RNASeq2GeneNorm
Interpolating... KIRC_mRNAArray
Interpolating... KIRP_RNASeq2GeneNorm
Interpolating... KIRP_mRNAArray
Interpolating... LGG_RNASeq2GeneNorm
Interpolating... LGG_mRNAArray
Interpolating... LUAD_RNASeq2GeneNorm
Interpolating... LUAD_mRNAArray
Interpolating... READ_RNASeq2GeneNorm
Interpolating... READ_mRNAArray
Interpolating... UCEC_RNASeq2GeneNorm
Interpolating... UCEC_mRNAArray


In [79]:
from umap import UMAP


Z = UMAP(n_components=2, spread = 2.0, min_dist=0.01, verbose=True).fit_transform(train_encoded)

UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=1.0, local_connectivity=1.0, metric='euclidean',
   metric_kwds=None, min_dist=0.01, n_components=2, n_epochs=None,
   n_neighbors=15, negative_sample_rate=5, random_state=None,
   repulsion_strength=1.0, set_op_mix_ratio=1.0, spread=2.0,
   target_metric='categorical', target_metric_kwds=None,
   target_n_neighbors=-1, target_weight=0.5, transform_queue_size=4.0,
   transform_seed=42, verbose=True)
Construct fuzzy simplicial set
Construct embedding
	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs


In [56]:
umap_train = UMAP(n_components=2, spread = 2.0, min_dist=0.01, verbose=True).fit_transform(X)

UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=1.0, local_connectivity=1.0, metric='euclidean',
   metric_kwds=None, min_dist=0.01, n_components=2, n_epochs=None,
   n_neighbors=15, negative_sample_rate=5, random_state=None,
   repulsion_strength=1.0, set_op_mix_ratio=1.0, spread=2.0,
   target_metric='categorical', target_metric_kwds=None,
   target_n_neighbors=-1, target_weight=0.5, transform_queue_size=4.0,
   transform_seed=42, verbose=True)
Construct fuzzy simplicial set
Construct embedding


/Users/kompa/anaconda3/lib/python3.6/site-packages/umap/spectral.py:229: UserWarning: Embedding a total of 4 separate connected components using meta-embedding (experimental)
  n_components


	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs


In [80]:
np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/embedding_385.csv', Z, delimiter=',')

In [57]:
np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/raw_embedding.csv', umap_train, delimiter=',')

In [53]:
np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/y_train.csv', y, delimiter = ',')
np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/y_test.csv', y_test, delimiter = ',')

In [55]:
np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/X_train.csv', X, delimiter=',')
np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/X_test.csv', X_test, delimiter=',')

In [82]:
from pykdtree.kdtree import KDTree

In [92]:
# KNN calculations 

kd_tree = KDTree(X)

In [93]:
brca_test_rna = translate_dataset(X_test, vae, 0)

In [94]:
dist, idx = kd_tree.query(brca_test_rna, k=10)

ValueError: Data and query points must have same dimensions

In [89]:
X.shape

(3943, 16146)

In [90]:
brca_test_rna.shape

(982, 16146)

In [95]:
from scipy.spatial import cKDTree

In [96]:
ckd_tree = cKDTree(X)

In [99]:

for idx, dataset in enumerate(microarray_datasets):
    print(idx, dataset)
    interolated_test_set = translate_dataset(X_test, vae, idx)
    dist, idx = ckd_tree.query(interolated_test_set, k=10)
    np.savetxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/'+dataset+'_NN.csv', idx)
    

0 BRCA_RNASeq2GeneNorm
1 BRCA_mRNAArray
2 COAD_RNASeq2GeneNorm
3 COAD_mRNAArray
4 KIRC_RNASeq2GeneNorm
5 KIRC_mRNAArray
6 KIRP_RNASeq2GeneNorm
7 KIRP_mRNAArray
8 LGG_RNASeq2GeneNorm
9 LGG_mRNAArray
10 LUAD_RNASeq2GeneNorm
11 LUAD_mRNAArray
12 READ_RNASeq2GeneNorm
13 READ_mRNAArray
14 UCEC_RNASeq2GeneNorm
15 UCEC_mRNAArray


In [100]:
test_idx = np.loadtxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/BRCA_RNASeq2GeneNorm_NN.csv')

In [105]:
for idx, dataset in enumerate(microarray_datasets):
    test_idx = np.loadtxt('/Users/kompa/Downloads/TCGA_mRNA_RNA/'+dataset+'_NN.csv')
    print(np.sum(test_idx==idx))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
